In [1]:
import numpy as np
from synd.core import load_model

Load the SynD model, and the source/target state definitions

In [2]:
model = load_model('model-lite.synd')
state_definitions = np.load('state_definitions.npz')

In [3]:
source_states = state_definitions['source']
target_states = state_definitions['target']

Seed the RNG for reproducible outputs

In [4]:
model.rng = np.random.default_rng(seed=5)

Generate a 1000-step trajectory, from our set of 30 source states

In [5]:
sample_trajectory = model.generate_trajectory(
    initial_states = source_states,
    n_steps = 1000,
)

Check that the sample trajectories actually reached the target state

In [6]:
np.isin(target_states, sample_trajectory).any()

True

In [7]:
np.savez('sample_trajectories.npz')